# Explorations and tests with the heavier data-star

- dst28gm06n052

st28gm06n052_186.sav

st28gm06n052_190.sav

st28gm06n052_198.sav

In [15]:
# Check stellar props
from scipy.io.idl import readsav
import numpy as np
import analyze_co5bold_functions as c5d

teststar = readsav('../co5bold_data/dst28gm06n052/st28gm06n052_186.sav')
teststar = teststar['ful']

# Star's props
print(teststar['PAR'][0][0][2])

#Grid's props
# Size of grid
print(np.shape(teststar['EOS'][0][0][0]))

# Number of cells along each axis
print(teststar['Z'][0][0][4])

# Minimum cellsize:
AUcm = 1.49598e13 # cm
c5dgrid,cellcourners,cellsize = c5d.load_grid_properties(
    savpath='../co5bold_data/dst28gm06n052/st28gm06n052_186.sav'
)

print(f'Minimum cell: {cellsize/AUcm} AU')
print(f'Total size of grid: {(np.max(cellcourners)-np.min(cellcourners))/AUcm} AU')

# '3D dusty AGB star model: 1 M_Sun, 355 R_Sun, L=7000 L_Sun'
# > Teff = 2800K



/usr/lib/python3/dist-packages/scipy/io/idl.py:469: UserWarning: Using experimental 64-bit array read
  warnings.warn("Using experimental 64-bit array read")


[b'Parameter file for RHD code:'
 b'3D dusty AGB star model: 1 M_Sun, 355 R_Sun, L=7000 L_Sun'
 b'Start model st28gm06n052_001.sta with 679^3 points'
 b'  based on st28gm06n050_300.end,'
 b'  extended inner and outer grid with rhd_manipulate.pro'
 b'Code version 2020-02-06:  MPI + OpenMP'
 b'Non-interacting forsterite+corundum dust, ar_dustrainabu: 3.0E-16'
 b'Non-grey 3-cluster opacity table:'
 b'  t2800gm050mm00_coma_opal_paola_oc1r_b3_n04.opta'
 b'"Big Box" buffer layer: N_radThinPoint=101+6, radThinHeatScheme=ConstantLum'
 b'Radiation pressure: radThinPressScheme=ConstantLum'
 b'From n034 on: c_radthintfactor=0.75->0.77']
(679, 679, 679)
[[-339  339]
 [-339  339]
 [-339  339]]
Minimum cell: 0.02582559459671921 AU
Total size of grid: 29.79639527908983 AU


In [17]:
# Create grid
import create_r3d_functions as c3d

# Smallest c5d cells are 0.026 AU
# Radius of star: 1.651AU (355 Rsun)
# Settings for the grid
AUcm = 1.49598e13 # cm
r3dedge = 29.796 * AUcm # Size of whole grid
basecell = 0.03*AUcm * 2**4 # Size of base cell (smallest cell is 0.03 AU)
nxyz = r3dedge/basecell
refinementlist_au = [9.5, 5.5, 3.5, 2.5]
refinementlist = [refs*AUcm for refs in refinementlist_au]

c3d.create_grid(
    gridedge=r3dedge/AUcm, 
    nxyz=nxyz, 
    refinementlist=refinementlist_au, 
    inrefine=0.13, 
    savegrid=True
)

# If I didn't have it already, create wavelength-grid here also!

Creating amr_grid with octree refinement.
Final length of total side of whole grid: 29.759999999999998 AU
Number of base cells along one side of the grid: 62
Distances to outer refinement limits from centrum: [9.5, 5.5, 3.5, 2.5] AU
Distances to inner refinement limits from centrum: [0.325, 0.65, 0.9750000000000001, 1.3] AU
Number refinements: 4
Size of base cell: 0.48 AU
Child cell size 1: 0.24 AU
Child cell size 2: 0.12 AU
Child cell size 3: 0.06 AU
Child cell size 4: 0.03 AU
Writing amr_grid.inp
Writing grid_distances.csv and grid_cellsizes.csv
(Not necessary for Radmc3d, but useful for pre/portprocessing of your model. They have the same order as dust_densities.inp)
Writing grid_info.txt
Create grid:
    amr_grid.inp
     grid_info.txt
    grid_distances.csv
    grid_cellsizes.csv
DONE



In [1]:
# Create onestar-data
# ie the not-binned data which treats the star as one specie

import analyze_co5bold_functions as a5d
import os

path = f'../r3dresults/st28gm06n052/'

for phase in ['186','190','198']:
    print(f'Doing phase: {phase}')
    
    a5d.create_star(
        savpath=f'../co5bold_data/dst28gm06n052/st28gm06n052_{phase}.sav',
        amrpath=path+'amr_grid.inp',
        gridpath=path+'grid_distances.csv',
        sizepath=path+'grid_cellsizes.csv'
    )

    print('Moving files')
    os.system(f'mv ../dust_density_onestar.inp {path}9species_{phase}/dust_density_onestar.inp')
    os.system(f'mv ../dust_temperature_onestar.dat {path}9species_{phase}/dust_temperature_onestar.dat')
    os.system(f'mv ../star_opacities.dat {path}9species_{phase}/star_opacities.dat')

print('All done')


Doing phase: 186
Loading R3D grid
Loading C5D grid properties


/usr/lib/python3/dist-packages/scipy/io/idl.py:469: UserWarning: Using experimental 64-bit array read
  warnings.warn("Using experimental 64-bit array read")


Loading C5D star properties (density, temperature, opacity)
Translating C5D data to R3D data
Finished 25 per cent of the grid.
Finished 50 per cent of the grid.
Finished 75 per cent of the grid.
C5D Dust-star:
    dust_density_onestar.inp
    dust_temperature_onestar.dat
    star_opacities.dat
DONE

Moving files
Doing phase: 190
Loading R3D grid
Loading C5D grid properties
Loading C5D star properties (density, temperature, opacity)
Translating C5D data to R3D data
Finished 25 per cent of the grid.
Finished 50 per cent of the grid.
Finished 75 per cent of the grid.
C5D Dust-star:
    dust_density_onestar.inp
    dust_temperature_onestar.dat
    star_opacities.dat
DONE

Moving files
Doing phase: 198
Loading R3D grid
Loading C5D grid properties
Loading C5D star properties (density, temperature, opacity)


KeyboardInterrupt: 

In [ ]:
# Create binned star data
import analyze_co5bold_functions as a5d
import os

nbins = 9
Teff = 2800

path = f'../r3dresults/st28gm06n052/'

for phase in ['186','190','198']:
    print(f'Doing phase: {phase}')

    a5d.create_staropacity(
        pathopacity = f'{path}9species_{phase}/star_opacities.dat',
        pathstardensity=f'{path}9species_{phase}/dust_density_onestar.inp',
        pathwavelength=f'{path}wavelength_micron.inp',
        pathtemperature=f'{path}9species_{phase}/dust_temperature_onestar.dat',
        temperaturelimit=5*Teff,
        nbins=nbins
    )

    print('Moving files')
    os.system(f'mv ../dust_density_starbins.inp {path}9species_{phase}/dust_density.inp')
    os.system(f'mv ../dustkappa_*.inp {path}9species_{phase}/')
    os.system(f'mv ../dustopac_starbins.inp {path}9species_{phase}/dustopac.inp')
    os.system(f'mv ../dust_temperature_starbins.dat {path}9species_{phase}/dust_temperature.dat')
    os.system(f'mv ../star_opacities_bins.dat {path}9species_{phase}/')

print('All done')


## Analyze input data

In [ ]:
# Plot densities as func of radius of all star-species
import analyze_r3d_functions as a3d

a3d.plot_alldensity_radius(
    path='../r3dresults/st28gm06n052/9species_186/'
)

In [ ]:
# Plot temperatures as func of radius of all star-species
import analyze_r3d_functions as a3d

a3d.plot_alltemperature_radius(
    path='../r3dresults/st28gm06n052/9species_186/'
)

In [ ]:
# Plot c5d's kappa in r3d-grid
import analyze_co5bold_functions as a5d

a5d.plot_opakapparadius(
    path='../r3dresults/st28gm06n052/9species_186/'
)


In [ ]:
# Plot all star-kappa as func of wavelength
import analyze_r3d_functions as a3d

a3d.plot_allkappa(
    path='../r3dresults/st28gm06n052/9species_186/'
)

## Analyze images and SED

In [ ]:
import analyze_r3d_functions as a3d
import matplotlib.pyplot as plt

path = '../r3dresults/st28gm06n052/9species_186/'

imagewaves = [1,10,1000]
images = [f'image_{imagewave}um.out' for imagewave in imagewaves]

testflux = a3d.plot_images(
    path=path,
    images=images,
    distance=1
)

maxflux,maxwave = a3d.plot_sed(path=f'{path}spectrum.out')

# Print luminosity in Lsols
# Wien's displacement law: freqmax = kb/h*T * 2.821439[..]
print(f"{a3d.compute_luminosity(path=f'{path}spectrum.out')/3.828e26} Lsol")
print(f'Max flux: {maxflux} Jy, at {maxwave} um.')
print(f'BB-temperature: {2.998e8/(5.879e10 * maxwave*1e-6)} K')

plt.plot(imagewaves,testflux,'r*')
